In [1]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import math
import operator
import collections
import ast

# For importing functions
import ipynb
import sys  

# Import data
sys.path.insert(0, 'C:/Users/Owner/Documents/Thesis/Code/Season_Analysis/Extraction_Partitioning_Functions')

from ipynb.fs.full.Season_Filter_Functions import djf_regions,jja_regions

In [11]:
both = pd.read_csv('C:/Users/Owner/Documents/Thesis/Code/Data_Extraction/McRobie_Data_Extraction/Raw_Data_Extraction/regional_mean_omega_all.csv')
both_extended = pd.read_csv('C:/Users/Owner/Documents/Thesis/Code/Data_Extraction/McRobie_Data_Extraction/Raw_Data_Extraction/regional_mean_omega_all_extended.csv')

In [3]:
def df_to_list(input_dataframe):
    dates = input_dataframe['Date'].tolist()
    nw_aus = input_dataframe['NW_Aus_Prec_Mean'].tolist()
    ne_aus = input_dataframe['NE_Aus_Prec_Mean'].tolist()
    mc = input_dataframe['MC_Prec_Mean'].tolist()
    io = input_dataframe['IO_Prec_Mean'].tolist()
    ism = input_dataframe['ISM_Prec_Mean'].tolist()
    
    easm = input_dataframe['EASM_Prec_Mean'].tolist()
    eep = input_dataframe['EEP_Prec_Mean'].tolist()

    return dates,nw_aus,ne_aus,mc,io,eep,ism,easm

def seasonal_indices(input_dataframe):
    index = []
    for i in range(len(input_dataframe)):
        index.append(i)
    
    djf_index = []
    for i in index:
        if i%12 == 0:
            djf_index.append(i) #January
        elif i%12 == 1:
            djf_index.append(i) # February
        elif i%12 == 11:
            djf_index.append(i) # December
            
    jja_index = []
    for i in index:
        if i%12 == 5:
            jja_index.append(i) #June
        if i%12 == 6:
            jja_index.append(i) # July
        if i%12 == 7:
            jja_index.append(i) # August
            
    return djf_index,jja_index


def extended_seasonal_indices(input_dataframe):
    index = []
    for i in range(len(input_dataframe)):
        index.append(i)
    
    ndjfm_index = []
    for i in index:
        if i%12 == 0:
            ndjfm_index.append(i) #January
        elif i%12 == 1:
            ndjfm_index.append(i) # February
        elif i%12 == 11:
            ndjfm_index.append(i) # December
        elif i%12 == 10:
            ndjfm_index.append(i) # November
        elif i%12 == 2:
            ndjfm_index.append(i) # March
            
    mjjas_index = []
    for i in index:
        if i%12 == 4:
            mjjas_index.append(i) #May
        if i%12 == 5:
            mjjas_index.append(i) #June
        if i%12 == 6:
            mjjas_index.append(i) # July
        if i%12 == 7:
            mjjas_index.append(i) # August
        if i%12 == 8:
            mjjas_index.append(i) # September
            
    return ndjfm_index,mjjas_index

In [40]:
def detrend_data(input_dataframe):
    from scipy import signal
    trended = df_to_list(input_dataframe)
    to_detrend = trended[1:]
    
    detrended = []
    for i in range(len(to_detrend)):
        time_series = to_detrend[i]
        detrend = signal.detrend(time_series)
        detrended.append(detrend)

    fuck_you = pd.DataFrame(list(zip(trended[0],
                                     detrended[0],
                                     detrended[1],
                                     detrended[2],
                                     detrended[3],
                                     detrended[4],
                                     detrended[5],
                                     detrended[6])),
                            columns = ['Date',
                                       'NW_Aus_Prec_Mean',
                                       'NE_Aus_Prec_Mean',
                                       'MC_Prec_Mean',
                                       'IO_Prec_Mean',
                                       'EEP_Prec_Mean',
                                       'ISM_Prec_Mean',
                                       'EASM_Prec_Mean'])
    
    return fuck_you
    

In [41]:
detrended_both = detrend_data(both)

In [16]:
both[0:50]

,Unnamed: 0,Date,NW_Aus_Prec_Mean,NE_Aus_Prec_Mean,MC_Prec_Mean,IO_Prec_Mean,EEP_Prec_Mean,ISM_Prec_Mean,EASM_Prec_Mean
0,0,01-01-1948,-0.014093,-0.010842,-0.012871,0.013100,-0.006069,0.003459,0.011909
1,1,01-02-1948,-0.006839,-0.005242,-0.003322,0.018733,-0.002046,0.001383,0.027109
2,2,01-03-1948,-0.049620,-0.011842,-0.005991,0.021039,0.009487,0.016998,0.012842
3,3,01-04-1948,-0.048584,0.002838,-0.014356,0.015284,0.003176,0.023289,0.020287
4,4,01-05-1948,-0.055202,0.003678,-0.012265,0.025569,0.012465,0.018622,0.037287
5,5,01-06-1948,-0.041584,0.007918,-0.011071,0.020671,-0.000113,0.015400,-0.003958
6,6,01-07-1948,-0.036893,0.008518,-0.010562,0.021978,0.003309,0.007357,0.009309
7,7,01-08-1948,-0.037111,-0.007162,-0.008796,0.017876,0.000376,0.008767,0.020109
8,8,01-09-1948,-0.026547,0.011278,-0.009351,0.015672,0.004398,0.006870,0.001998
9,9,01-10-1948,-0.016147,-0.011682,-0.005568,0.021304,0.008909,0.002989,0.026687


In [35]:
def djf_jja_0(input_dataframe):
    djf_indices,jja_indices = seasonal_indices(input_dataframe)

    dates = df_to_list(input_dataframe)[0]
    djf_region_time_series = df_to_list(input_dataframe)[1:-2]
    jja_region_time_series = df_to_list(input_dataframe)[-2:]
    
    djf_filtered = []
    for i in range(len(djf_region_time_series)):
        region_time_series = djf_region_time_series[i]
        filtered_region_djf = []
        for j in range(len(region_time_series)):
            actual_index = j
            if actual_index in djf_indices:
                #print(region_time_series[actual_index])
                filtered_region_djf.append(region_time_series[actual_index])
            else:
                #print(0)
                filtered_region_djf.append(int(0))
        djf_filtered.append(filtered_region_djf)
        
    jja_filtered = []
    for i in range(len(jja_region_time_series)):
        region_time_series = jja_region_time_series[i]
        filtered_region_jja = []
        for j in range(len(region_time_series)):
            actual_index = j
            if actual_index in jja_indices:
                #print(region_time_series[actual_index])
                filtered_region_jja.append(region_time_series[actual_index])
            else:
                #print(0)
                filtered_region_jja.append(int(0))
        jja_filtered.append(filtered_region_jja)
    
    combined_dataframe = pd.DataFrame(list(zip(dates,
                                               djf_filtered[0],
                                               djf_filtered[1],
                                               djf_filtered[2],
                                               djf_filtered[3],
                                               djf_filtered[4],
                                               jja_filtered[0],
                                               jja_filtered[1])),
                                      columns = ['Date',
                                                 'NW_Aus_Mean_Vorticity',
                                                 'NE_Aus_Mean_Vorticity',
                                                 'MC_Mean_Vorticity',
                                                 'IO_Mean_Vorticity',
                                                 'EEP_Mean_Vorticity',
                                                 'ISM_Aus_Mean_Vorticity',
                                                 'EASM_Mean_Vorticity'])
    
    return combined_dataframe

In [42]:
detrended_djf_jja_0 = djf_jja_0(detrended_both)

In [43]:
detrended_djf_jja_0

,Date,NW_Aus_Mean_Vorticity,NE_Aus_Mean_Vorticity,MC_Mean_Vorticity,IO_Mean_Vorticity,EEP_Mean_Vorticity,ISM_Aus_Mean_Vorticity,EASM_Mean_Vorticity
0,01-01-1948,0.000874,-0.008912,0.004816,0.006782,-0.011215,0.0,0.0
1,01-02-1948,0.008116,-0.003332,0.014348,0.012391,-0.007178,0.0,0.0
2,01-03-1948,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
3,01-04-1948,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
4,01-05-1948,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...
789,01-10-2013,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
790,01-11-2013,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0
791,01-12-2013,-0.067969,-0.026724,-0.019166,-0.006633,-0.003404,0.0,0.0
792,01-01-2014,-0.034581,0.009375,-0.019286,-0.003534,-0.006945,0.0,0.0


In [11]:
both_djf_jja_0.to_csv('land_ocean_djf_jja_0_mean_omega.csv')

In [45]:
detrended_djf_jja_0.to_csv('land_ocean_djf_jja_0_mean_omega_detrended.csv')

In [7]:
def ndjfm_mjjas_0(input_dataframe):
    ndjfm_indices,mjjas_indices = extended_seasonal_indices(input_dataframe)

    dates = df_to_list(input_dataframe)[0]
    ndjfm_region_time_series = df_to_list(input_dataframe)[1:-2]
    mjjas_region_time_series = df_to_list(input_dataframe)[-2:]
    
    ndjfm_filtered = []
    for i in range(len(ndjfm_region_time_series)):
        region_time_series = ndjfm_region_time_series[i]
        filtered_region_ndjfm = []
        for j in range(len(region_time_series)):
            actual_index = j
            if actual_index in ndjfm_indices:
                #print(region_time_series[actual_index])
                filtered_region_ndjfm.append(region_time_series[actual_index])
            else:
                #print(0)
                filtered_region_ndjfm.append(int(0))
        ndjfm_filtered.append(filtered_region_ndjfm)
        
    mjjas_filtered = []
    for i in range(len(mjjas_region_time_series)):
        region_time_series = mjjas_region_time_series[i]
        filtered_region_mjjas = []
        for j in range(len(region_time_series)):
            actual_index = j
            if actual_index in mjjas_indices:
                #print(region_time_series[actual_index])
                filtered_region_mjjas.append(region_time_series[actual_index])
            else:
                #print(0)
                filtered_region_mjjas.append(int(0))
        mjjas_filtered.append(filtered_region_mjjas)
    
    combined_dataframe = pd.DataFrame(list(zip(dates,
                                               ndjfm_filtered[0],
                                               ndjfm_filtered[1],
                                               ndjfm_filtered[2],
                                               ndjfm_filtered[3],
                                               ndjfm_filtered[4],
                                               mjjas_filtered[0],
                                               mjjas_filtered[1])),
                                      columns = ['Date',
                                                 'NW_Aus_Mean_Omega',
                                                 'NE_Aus_Mean_Omega',
                                                 'MC_Mean_Omega',
                                                 'IO_Mean_Omega',
                                                 'EEP_Mean_Omega',
                                                 'ISM_Aus_Mean_Omega',
                                                 'EASM_Mean_Omega'])
    
    return combined_dataframe

In [12]:
both_ndjfm_mjjas_0 = ndjfm_mjjas_0(both_extended)
both_ndjfm_mjjas_0.to_csv('land_ocean_ndjfm_mjjas_0_mean_omega.csv')

# Removing Zeros

In [48]:
def djf_jja(input_dataframe):
    djf_indices,jja_indices = seasonal_indices(input_dataframe)

    dates = df_to_list(input_dataframe)[0]
    djf_region_time_series = df_to_list(input_dataframe)[1:-2]
    jja_region_time_series = df_to_list(input_dataframe)[-2:]
    
    djf_filtered = []
    djf_dates = []
    for i in range(len(djf_region_time_series)):
        region_time_series = djf_region_time_series[i]
        filtered_region_djf = []
        for j in range(len(region_time_series)):
            actual_index = j
            if actual_index in djf_indices:
                #print(region_time_series[actual_index])
                filtered_region_djf.append(region_time_series[actual_index])
                djf_dates.append(dates[actual_index])
            else:
                pass
        djf_filtered.append(filtered_region_djf)
        
    jja_filtered = []
    jja_dates = []
    for i in range(len(jja_region_time_series)):
        region_time_series = jja_region_time_series[i]
        filtered_region_jja = []
        for j in range(len(region_time_series)):
            actual_index = j
            if actual_index in jja_indices:
                #print(region_time_series[actual_index])
                filtered_region_jja.append(region_time_series[actual_index])
                jja_dates.append(dates[actual_index])
            else:
                pass
        jja_filtered.append(filtered_region_jja)
    
    djf_dataframe = pd.DataFrame(list(zip(djf_dates,
                                          djf_filtered[0],
                                          djf_filtered[1],
                                          djf_filtered[2],
                                          djf_filtered[3],
                                          djf_filtered[4])),
                                 columns = ['Date',
                                            'NW_Aus_Mean_Vorticity',
                                            'NE_Aus_Mean_Vorticity',
                                            'MC_Mean_Vorticity',
                                            'IO_Mean_Vorticity',
                                            'EEP_Mean_Vorticity'])
    
    jja_dataframe = pd.DataFrame(list(zip(jja_dates,
                                          jja_filtered[0],
                                          jja_filtered[1])),
                                 columns = ['Date',
                                            'ISM_Mean_Vorticity',
                                            'EASM_Mean_Vorticity'])
    
    return djf_dataframe,jja_dataframe

In [49]:
djf,jja = djf_jja(both)

In [54]:
djf
#djf.to_csv('land_ocean_djf_mean_omega.csv')

,Date,NW_Aus_Mean_Vorticity,NE_Aus_Mean_Vorticity,MC_Mean_Vorticity,IO_Mean_Vorticity,EEP_Mean_Vorticity
0,01-01-1948,-0.014093,-0.010842,-0.012871,0.013100,-0.006069
1,01-02-1948,-0.006839,-0.005242,-0.003322,0.018733,-0.002046
2,01-12-1948,-0.033075,-0.001722,-0.003951,0.001263,-0.000847
3,01-01-1949,-0.033002,-0.012242,-0.001288,0.010304,0.004909
4,01-02-1949,-0.024984,-0.013122,-0.004048,0.008692,0.013331
...,...,...,...,...,...,...
195,01-01-2013,-0.045438,-0.017162,-0.023596,0.015671,0.003309
196,01-02-2013,-0.056347,-0.005402,-0.022271,0.018100,0.000331
197,01-12-2013,-0.073366,-0.012522,-0.023431,0.018182,-0.010091
198,01-01-2014,-0.039966,0.023598,-0.023534,0.021304,-0.013647


In [55]:
jja
#jja.to_csv('land_ocean_jja_mean_omega.csv')

,Date,ISM_Mean_Vorticity,EASM_Mean_Vorticity
0,01-06-1948,0.015400,-0.003958
1,01-07-1948,0.007357,0.009309
2,01-08-1948,0.008767,0.020109
3,01-06-1949,0.002724,0.009798
4,01-07-1949,-0.015181,0.000198
...,...,...,...
193,01-07-2012,0.013733,0.009109
194,01-08-2012,-0.002515,-0.017491
195,01-06-2013,-0.001934,0.012065
196,01-07-2013,-0.006746,0.008153
